In [ ]:
import os
import sys
import pymongo
from time import sleep
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# splitting data
from sklearn.model_selection import train_test_split

# Features Extraction
from sklearn.feature_extraction.text import CountVectorizer

# Evaluation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

# models
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC


In [ ]:
sys.path.append(os.path.abspath('../scraping_cleaning'))
from cleaning_data import *

In [ ]:
df_positive = pd.read_csv('../csv_files/positive_reviews.csv')
df_negative = pd.read_csv('../csv_files/negative_reviews.csv')
df_positive = df_positive.iloc[:8000]

In [ ]:
df_negative.dropna(inplace=True)
df_positive.dropna(inplace=True)

In [ ]:
len(df_negative)

In [ ]:
len(df_positive)

In [ ]:
df_negative['polarity'] = 0
df_positive['polarity'] = 1

In [ ]:
df_positive = df_positive[['positive_review', 'polarity']]
df_negative = df_negative[['negative_review', 'polarity']]

In [ ]:
sys.path.append(os.path.abspath('../file_handles'))
from handle_combind_shuffle_reviews import *

In [ ]:
df_positive = df_positive.rename(columns={'positive_review': 'reviews'})
df_negative = df_negative.rename(columns={'negative_review': 'reviews'})

In [ ]:
df_positive.head()

In [ ]:
df_negative.head()

In [ ]:
combined_reviews = combine_positive_negative_reviews(df_positive, df_negative)
cleaned_reviews = list(combined_reviews['reviews'])
# cleaned_reviews =list(cleaned_reviews)

In [ ]:
cleaned_reviews = arabic_pip_line(cleaned_reviews)

In [ ]:
combined_reviews['reviews'] = cleaned_reviews

In [ ]:
combined_reviews.head()

In [ ]:
df = shuffle_dataframe_of_reviews(combined_reviews)

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
len(df)

In [ ]:
X = df['reviews'] 
y = df['polarity']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1)
X_train[:5]

In [ ]:
print("Our training data now are: " + str(len(X_train))  + " Reviews")
print("Our testing data now are: " + str(len(X_test))  + " Reviews")
print("Our training data now are: " + str(len(y_train))  + " labels")
print("Our testing data now are: " + str(len(y_test))  + " labels")

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf_vectorizer(df):
    '''
    Argumen:
        df dataframe of multiple reviews
    return:
        Train & test arrays that can fir to the model
    '''
# I fit the vector to all of the data
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_vectorizer = tfidf_vectorizer.fit(X) 
    word_idf_weights = tfidf_vectorizer.idf_
    print("Our 10 words weights\n\n",word_idf_weights[:10])
# fit splited data
    testing_data = tfidf_vectorizer.transform(X_test)
    training_data = tfidf_vectorizer.transform(X_train) 
# convert to array that can apply to ML model
    training_data = training_data.toarray()
    testing_data = testing_data.toarray()
    return training_data, testing_data

In [ ]:
training_data, testing_data = tfidf_vectorizer(df)

In [ ]:
# first shape is the data itself and second shape is the BOW in our data
print("Our new vectorized data: " + str(training_data.shape))
print("Our new vectorized data: " + str(testing_data.shape)) 
print("The first 2 review after transform: \n", testing_data[:2])

In [ ]:
clf_MultinomialNB = MultinomialNB()

In [ ]:
model = clf_MultinomialNB.fit(training_data, y_train)

In [ ]:
predict = model.predict(training_data)

In [ ]:
print("F1 score of our training data is: ", f1_score(y_train, predict, average='micro'))

In [ ]:
print("Evalution Matrix of training data is \n", confusion_matrix(y_train, predict))

In [ ]:
predict = model.predict(testing_data)

In [ ]:
print("F1 score of our testing data is: ", f1_score(y_test, predict, average='micro'))

In [ ]:
print("Evalution Matrix of training data is \n", confusion_matrix(y_test, predict))

In [ ]:
clf_LogisticRegression = LogisticRegression(penalty='l2', tol=0.00001, solver='liblinear',max_iter=1000)

In [ ]:
logistic_model = clf_LogisticRegression.fit(training_data, y_train)

In [ ]:
predict = logistic_model.predict(training_data)

In [ ]:
print("F1 score of our testing data is: ", f1_score(y_train, predict, average='micro'))

In [ ]:
print("Evalution Matrix of training data is \n", confusion_matrix(y_train, predict))

In [ ]:
predict = logistic_model.predict(testing_data)

In [ ]:
print("F1 score of our testing data is: ", f1_score(y_test, predict, average='micro'))

In [ ]:
print("Evalution Matrix of training data is \n", confusion_matrix(y_test, predict))

In [ ]:
clf_SVC = SVC(kernel='linear')

In [ ]:
svc_model = clf_SVC.fit(training_data, y_train)

In [ ]:
predict = svc_model.predict(training_data)

In [ ]:
print("F1 score of our testing data is: ", f1_score(y_train, predict, average='micro'))

In [ ]:
print("Evalution Matrix of training data is \n", confusion_matrix(y_train, predict))